<a href="https://colab.research.google.com/github/lharikumar/OpenAI_FunctionCalling/blob/main/ExternalAPI_FC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Pre-requisites

1. Get the OpenAI API key at https://platform.openai.com/account/api-keys
2. Get the weather API key from any website that provides weather APIs. I got the key from https://home.openweathermap.org/api_keys

### Install and import the required libraries



In [1]:
!pip install -qU openai python-dotenv

In [2]:
import requests
import dotenv
import os
import openai
import json

### Read the keys from the .env files

**Never** use you API keys directly in the code

In [3]:
## Read the keys from the .env file
dotenv.load_dotenv("./env_files/.env")
weather_api_key = os.getenv('WEATHER_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

### The Weather API

**Note** - The API outputs the temperature in Kelvin

In [4]:
def get_current_weather(location, unit):
    location = location.split(',')[0]
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&unit={unit}"


    try:
        response = requests.get(url)
        response.raise_for_status()
        weather_data = response.json()


        converstion_factor = 0
        if unit == "imperial":
          converstion_factor = 33.8

        weather_info = {
            "location": location,
            "temperature": (weather_data["main"]["temp"] - 273.15) + converstion_factor,
            "unit": unit
        }
        return json.dumps(weather_info)
    except requests.exceptions.RequestException as e:
        print(f"Error calling the weather API: {e}")

In [5]:
resp = get_current_weather("Bangalore", "celsius")
resp

'{"location": "Bangalore", "temperature": 21.900000000000034, "unit": "celsius"}'

# OpenAI Function Calling

## Step 1 - Call the model with functions and the user’s input

In [6]:
available_functions = {
            "get_current_weather": get_current_weather,
        }

In [7]:
openai.api_key = os.getenv("OPENAI_API_KEY")
messages = [{"role": "user", "content": "What's the weather like in Boston ?"}]
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The  unit system for temperature used at location",
                     },
                },
                "required": ["location", "unit"],
            },
        }
    ]

response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto"
    )

print(response.choices[0].message)


{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\",\n  \"unit\": \"imperial\"\n}"
  }
}


## Step 2 - Use the model response to call your API

In [8]:
response_msg = response.choices[0].message
if response_msg.get("function_call"):
    function_name = response_msg["function_call"]["name"]
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_msg["function_call"]["arguments"])
    function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit")
    )

In [9]:
response_msg

<OpenAIObject at 0x7fe48b5bec00> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\",\n  \"unit\": \"imperial\"\n}"
  }
}

In [10]:
function_response

'{"location": "Boston", "temperature": 54.08000000000003, "unit": "imperial"}'

## Step 3 - Send the response back to the model to summarize

In [11]:
messages.append(response_msg)  # extend conversation with assistant's reply
messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
response_to_user = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response

In [12]:
response_to_user

<OpenAIObject chat.completion id=chatcmpl-7ULOYviZ0twQMZFLJxSyNJzpCeXwl at 0x7fe48b7afba0> JSON: {
  "id": "chatcmpl-7ULOYviZ0twQMZFLJxSyNJzpCeXwl",
  "object": "chat.completion",
  "created": 1687466438,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston, MA is 54.08\u00b0F."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 75,
    "completion_tokens": 14,
    "total_tokens": 89
  }
}